# 7. 합성공 신경망(CNN)
>
    > 이번장 주제는 합성곱 신경망 이다.  
    CNN은 이미지 인식과 음성 인식등 다양한 곳에서 사용되는데,  
    특히 이미지 인식 분야에서 딥러닝 활요한 기법은 거의 다 CNN을 기초로 한다.  
    이번장에서는 이 기술의 메커니즘을 자세히 설명하고 이를 파이썬으로 구현해보겠다.  

## 7-1 전체 구조
>
    > 지금까지 본 신경망은 인접하는 계층의 모든 뉴런과 결합되어 있었다.  
    이를 완전연결(fully-connected, 전결합)이라고 하며  
    완전히 연결된 계층을 Affine 계층이라는 이름으로 구현했다.
    Affine 계층ㅇ을 사용하면, 가령ㅇ 층이 5개인 전결합 신경망은 그림 7-1과 같이 구현 할 수 있다.  

>
    > 그림 7-1과 같이 전결합 신경망은 Affine 계층 뒤에 활성화 함수를 갖는 ReLU계층이 이어진다.  
    이 그림에서는 Affine-ReLU 조합이 4개가 쌓였고  
    마지막 5번째 층은 Affine계층에 이어 소프트맥스 계층에서 최종 결과를 출력한다. 그럼 합성곱은 어떻게 다를까?

>
    > 그림 7-2가 바로 cnn의 예이다.  
    그림과 같이 CNN에서는 새롱누 합성곱 계층과 풀링 계층이 추가가 된다. 
    CNN계층의 흐름은 Conv-ReLU-(Pooling) 과 같이 연결된다.  

>
    > 또다른 주목할 점은 출력에 가까운 층에서는 지금까지의 Affine-ReLU구성을 사용할수 있다는 것이다.  
    또, 마지막 출력 계층에서는 Affine-Softmax조합을 그대로 사용한다.  
    이상은 일반적인 CNN에서 흔히 볼 수 있는 구성이다. 

## 7-2 합성공 계층
>
    > CNN에서는 padding,stride등 CNN고유의 용어가 등장한다.  
    또, 각 계층 사이에는 3차원 데이터 같이 입체적인 데이터가 흐른다는 점에서 완전연결 신경망과는 다르다.  
    우선 이번 절 에서는 CNN에서 사용한 합성곱 계층의 구조를 차분히 살펴보기로 하겠다.

### 7-2.1 합성공 계층
>
    > 지급까지는 전연결 계층을 사용하였다.  
    완전연결 계층에서는 인접하는 계층의 뉴런이 모두 연결 되고 출력의 수는 임의로 정할 수 있었다.  

>
    > 완전 연결 계층의 문제는 무엇일까?  
    바로 데이터의 형상이 무시된다는 사실이다.  
    데이터가 이미지인 경우를 예를 들어보자.  
    이미지는 통상 세로 가로 채널로 구성된 3차원 데이터 이다.  
    그러나 완전 연결 계층에 입력할 땐느 3차원 데이터를 평평한 1차원 데이터로 평탄화 해줘야 한다.  
    사실 지금까지 MNIST 데이터셋을 사용한 사례에서는 형상이 (1,28,28)인 이미지를  
    1줄로 세운 784개의 데이터로 변환하여 입력하였다.

>   
    > 이미지는 3차원 형상이며, 이형상에는 소중한 공간적 정보가 담겨있다.  
    예를 들어 공간적으로 가까운 픽셀은 값이 비슷하거나, RGB의 각 채널은 서로 밀접한 관련이 되어있거나  
    거리가 먼 픽셀끼리는 별 연관이 없는 등, 3차원 속에서 의미를 가즌ㄴ 본질적인 패턴이 숨어있을 것이다.  
    그러나 완전연결 계층은 형상을 무시하고 모든 입ㄹ력 데이터를 동등한 뉴런으로 취급하여 형상에 담긴 정보를 살릴수 없다.  
>
    >CNN에서는 합성공 계층의 입출력 데이터를 특징맵이라고도 부른다.(feature map)  
    합성곱 계층의 입력 데이터를 입력 특징 맵(input feature map),  
    출력 데이터를 출력특징 맵(output feature map)이라고 하는 식이다.  
    이 책에서는 입출력 데이터와 특징 맵을 같은 의미로 사용한다.

### 7-2.2 합성곱 연산  
>
    > 합성곱 계층에서의 합성곱 연산을 처리한다.  
    합성곱 연산은 이미지 처리에서 말하는 필터 연산에 해당한다.  
    구체적인 예를 보며 이해해보자(그림 7-3)  
    이 예시에서 입력 데이터는 세로 가로 방향의 형상을 가졌고, 필터 역시 세로 가로 방향의 차원을 가진다.  
    데이터와 필터의 형상을 (높이,너비)로 표기하며  
    이예에서는 입력은 (4,4) 필터는 (3,3) 출력은 (2,2)가 된다.  
    문헌에 따라 필터를 커널이라 칭하기도 한다.  
>
    > 그럼 그림 7-3의 합성곱 연산 예에서 어떤 계산이 이뤄지는지 설명하겠다.  
    그림 7-4는 이 합성곱 연산의 계산 순서를 그려본 것이다.  

>
    >합성곱 연산은 필터의 윈도우를 일정 간격으로 이동해가며 입력 데이터에 적용한다.  
    여기에서 말하는 윈도우는 그림 7-4의 회색 3x3부분을 가리킨다.  
    이 그림에서 보듯 입력과 필터에서 대응하는 원소끼리 곱한 후 그 총합을 구한다.(이 계산은 단일 곱셈-누산(fused multiply -add))  
    이 과정에서 모든 장소에서 수행하면 합성공 연산의 출력이 완성된다.

>
    > 자, 완전연결 신경망에는 가중치 매개변수와 편향이 존재하는데, CNN에서는 필터의 매개변수가 그동안의 '가중치'에 해당한다.  
    그리고 CNN에서도 편향이 존재한다. 그림 7-3은 필터를 적용하는 단계까지만 보여준 것이고, 편향까지 포함하면 그림 7-5와 같은 흐름이 된다.  
    그리고 편향은 항상 1x1 만 존재한다. 그 하나의 값을 필터를 적용한 모든 원소에 더하는 것이다.  

### 7-2.3 Padding
>
    > 합성곱 연산을 수행하기 전에 입력 데이터 주변을 특정 값으로 채우기도 한다.  
    이를 패딩이라 하며, 합성곱 연산에서 자주 이용하는 기법이다.  
    예를 들어 그림 7-6은 (4,4) 크기의 입력 데이터에 폭이 1인 패딩을 적용한 모습이다.  
    입력 데이터 사방 1픽셀을 특정 값으로 채우는 것이다.  

>
    > 그림과 같이 처음에 크기가 (4,4)인 입력 데이터에 패딩이 추가되어 (6,6)이 된다.  
    이 입력에 (3,3) 크기의 필터를 걸면 (4,4) 크기의 출력 데이터가 생성된다.  
    이 예에서는 패딩을 1로 설정했지만,2나 3등 원하는 정수로설정 할 수 있다. 만약 그림 7-5에 패딩을 2로 설정하면  
    입력데이터의 크기는 (8,8)이 되고 3으로 설정하면 (10,10)이 된다.

### 7-2.4 Stride
>
    >  필터를 적용하는 위치의 간격을 스트라이드 라고 한다.  
    지금까지 본 예는 모두 스트라이드가 1이었지만  
    예를 들어 스트라이드를 2로 하면 필터를 적용하는 윈도우가 두 칸씩 이동한다.(그림 7-7)

>
    > 그림 에서는 크기가 (7,7)인 입력 데이터에 스트라이드를 2로 설정한 필터를 적용한다.  
    이처럼 스트라이드는 필터를 적용하는 간격을 지정한다.  

>
    > 그런데 스트라이드를 2로 하니 출력은 (3,3)이 된다.  
    이처럼 스트라이드를 키우면 출력 크기는 작아진다.  
    한편 패딩을 크게하면 출력 크기가 커졌다.  
    이러한 관계를 수식화 하면 식 7.1이 된다.  
    입력 크기를 (H,W) 필터 크기를 (FH,FW), 출력 크기를 (OH,OW), 패딩을 P, 스트라이드를 S라 하면 출력 크기는 다음식으로 계산한다.
    단 주의할점은 출력크기가 정수로 나눠떨어지는 값이어야 한다는 점에 주의해야한다. 출력 크기가 정수가 아니면 오류를 내는 등의 대응을 해야한다.  

### 7-2.5 3차원 데이터의 합성곱 연산

>   
    > 지금까지 2차원 형상을 다루는 합성곱 연산을 살펴보았다.  
    그러나 이미지만 해도 세로, 가로에 더해서 채널까지 고려한 3차원 데이터 이다.   
    이번절 에서는 조금 전과 같은 순서로 , 채널까지 고려한 3차원 데이터를 다루는 합성공 연산을 살펴보겠다.  

>   
    > 그림 7-8은 3차원 데이터의 합성곱 연산 예이다. 그리고 그림 7-9는 계산 순서이다.  
    2차원일때와 비교하면 길이 방향으로 특징 맵이 늘어나 ㅆ다. 채널 쪽으로 특징 맵이 려러개 있다면  
    입력 데이터와 필터의 합성곱 연산을 채널마다 수행하고 그 겨로가를 더해서 하나로 출력을 얻는다.  
    3차원 합성곱 연산에서 주의할 점은 입력 데이터의 채널 수와 필터의 채널수가 같아야 한다는 것이다.  

### 7-2.6 블록으로 생각하기

>
    > 3차원 합성곱 연산은 데이터와 필터를 직육면체 블록이라고 생가하면 쉽다.  
    블록은 3차원 직육면체이다. 또 3차원 데이터를 다차원 배열로 나타낼 때는 (채널,높이,너비) 순서로 쓰겠다.  
    예를 들어 채널수 C, 높이 H, 너비 W인 데이터 형상은 (C,H,W)로 쓴다.  
    필터도 같은 순서로 쓴다. (C,FH,FW)로 쓴다.

>
    > 자 이 예에서 출력 데이터는 한장의 특징 맵이다. 한장의 특징 맵을 다른 말로 하면 채널이 1개인 특징 맵이다. 
    그럼 합성곱 연산의 출력으로 다수의 채널을 내보내려면 어떻게 해야 할까?  
    그 답은 필터(가중치)를 사용하는 것이다. 그림으로는(그림 7-11) 처럼 된다.

>
    > 이그림과 같이 필터를 FN개 적용하면 출력맵도 FN개가 생성된다.  
    그리고 그 FN개의 맵을 모으면 형상이 (FN,OH,OW)인 블록이 완성됩니다.  
    이 완성된 블록을 다음 계층으로 넘기겠다는 것이 CNN의 처리 흐름이다.

>
    > 이상에서 보든 합성곱 연산에서는 필터의 수도 고려해야 한다.  
    그런 이유로 필터의 가중치를 4차원 데이터이며 (출력 채널수, 입력 채널수, 높이, 너비)순으로 쓴다.  
    예를 들어 채널 수 3 (5,5)인 필터가 20개가 있다면 (20,3,5,5)로 쓴다.  
    자 합성곱 연산에도 편향이 쓰인다. 
    그림 7-12는 그림 7-11에 편향을 더한 모습이다.

### 7-2.7 배치처리

>
    >신경망 처리에서는 입력 데이터를 한 덩어리로 묶어 배치로 처리했다.  
    전연결 신경망을 구현하면서는 이 방식을 지원하여 처리 효율을 높이고, 미니 배치 방식의 학습도 지원하도록 하겠다.  
    그래서 데이터 형식은 다음과 같다, (데이터수, 채널수, 높이, 너비) 
    데이터가 N개일때 그림 7-12를 배치 처리한다면 데이터 형태가 그림 7-13 처럼 되는 것이다.  

## 7-3 풀링 계층

>
    > 풀링은 세로 가로 방향의 공간을 줄이는 연산이다. 예를 들어 그림 7-14 와 같이 2x2 영역을 원소 하나로 집약하여 공간 크기를 줄인다.
    그림은 max pooling을 스트라이드 2로 처리한 순서이다. max pooling은 그림과 같이  
    2x2크기의 영역에서 가장 큰 원소 하나를 꺼낸다. 또 스트라이드는 이 예에서는 2로 설정했으므로 
    2x2 윈도우가 원소 2칸 간격으로 이동한다.  
    참고로 풀링의 윈도우 크기와 스트라디는 같은 값으로 설정하는 것이 보통이다.  

### 7-3.1 풀링 계층의 특징
>
    > 풀링 계층의 특징은 다음과 같다.

> - 학습해야할 매개변수가 없다.
>
    > 풀링 계층은 합성곱 계층과 달리 학습해야할 매개변수가 없다. 풀링은 대상 여역에서 최댓값이나 평균을 취하는 처리이므로 특별히 학습할 것이 없다.  
> - 채널 수가 변하지 않는다.  
>
    > 풀링 연산은 입력 데이터의 채널수 그대로 출력 데이터로 내보낸다. 그림 처럼 채널마다 독립적으로 계산하기 때문이다.
> - 입력의 변화에 영향을 적게 받는다.
>
    > 입력 데이터가 조금 변해도 풀리의 결과는 잘 변하지 않는다. 
    예를 들어 그림 7-16은 입력 데이터의 차이를 풀링이 흡수해 사라지게 하는 모습을 보여준다.




## 7.4 합성곱/풀링 계층 구현하기

>
    > 앞에서 설명한 대로 CNN에서 계층 사이를 흐르는 데이터는 4차원 데이터이다.  
    예를 들어 데이터의 형상이 (10,1,28,28)이라면,  
    이는 높이 28 너비 28 채널 1개인 데이터가 10개 라는 이야기다. 이를 파이썬으로 구현하면 다음과 같다.

In [1]:
import numpy as np

x = np.random.rand(10,1,28,28)
print(x.shape)

(10, 1, 28, 28)


여기에서 10개 중 첫 번째 데이터에 접근하려면 단순히 x[0]이라고 작성한다.  
마찬가지로 두 번째 데이터는 x[1]에 위치하여 있다.

In [2]:
print(x[0].shape)
print(x[1].shape)

(1, 28, 28)
(1, 28, 28)


또한 첫번째 데이터의 첫 채널의 공간 데이터에 접근하려면 다음과 같이 작성한다.

In [3]:
print(x[0,0])

[[0.61817174 0.69433764 0.60454504 0.84350869 0.17663368 0.00686315
  0.35258664 0.08770597 0.95008219 0.00296476 0.02871343 0.67553134
  0.76459481 0.44129398 0.80390751 0.57365407 0.68820943 0.02922827
  0.00208787 0.41959007 0.37673642 0.89575764 0.806549   0.79317398
  0.45763185 0.58939121 0.5431789  0.24016778]
 [0.4868519  0.89396533 0.5928409  0.69457211 0.88264255 0.4433112
  0.65755464 0.83653493 0.2237577  0.93859284 0.27504641 0.14037969
  0.84899918 0.18738683 0.01873146 0.20884873 0.94612433 0.55447265
  0.6050514  0.91125022 0.62928042 0.59179223 0.19879107 0.56523826
  0.13846753 0.80828271 0.70757982 0.9179279 ]
 [0.94082468 0.72025765 0.79367882 0.65707038 0.74944256 0.75764852
  0.75468819 0.41997975 0.45966631 0.98127027 0.30855443 0.99842112
  0.46856248 0.81451095 0.54727718 0.89864973 0.89032938 0.71829593
  0.33031282 0.88811807 0.93938757 0.67424504 0.35792936 0.84811064
  0.74020543 0.26319049 0.46373762 0.83106468]
 [0.25229539 0.61645227 0.5455258  0.1449659

### 7-4.2 im2col로 데이터 전개하기
>
    > 합성곱 연산을 곧이곧대로 구현하려면 for 문을 겹겹이 써야한다.  
    생각만 해도 귀찮고 또 넘파이에 for문을 사용하면 성능이 떨어진다는 단점이 있다.  
    이번 절 에서는 for문 대신 im2col이라는 편의함수를 사용하여 간단하게 구현해보겠다.  

>
    > im2col은 입력 데이터를 필터링(가중치 계산)하기 좋게 전개하는 함수이다.  
    그림 7-17과 같이 3차원 입력 데이터에 im2col을 적용하면 2차원 행렬로 바뀐다(정확히는 배치 안의 데이터 수까지 포함한 4차원 데이터를 2차원으로)  
    im2col은 필터링 하기 좋게 입력 데이터를 전개한다. 구체적으로는 그림 7-18과 같이 
    입력 데이터에서 필터를 적용하는 영역을 한줄로 늘어놓는다.  
    이 전개를 필터를 적용하는 모든 영역에 대해 수행하는데 im2col이다.  

>
    > 그림 7-18에는 보기 좋게끔 스트라이드를 크게 잡아 피러틩 적용 영역이 겹치지 않도록 했지만  
    실제 상황에서는 영역이 겹치는 경우가 대부분이다. 필터 적용 영역이 겹치게 되면  
    im2co로 전개한 후 원소 수가 원래 블록의 원소 수 보다 많아진다.  
    그래서 im2col을 사용해 구현하며 메모미를 더 많이 소비한다는 단점이 있다.  
    하지만 컴퓨터는 큰 행령을 묶어서 계산하는데 탁월하다.  
    예를 들어 행렬 계산 라이브러러 등은 행렬 계산에 고도로 최적화되어 큰 행렬의 곱셈을 빠르게 계산 가능하다.  
    그래서 문제를 행렬 계산으로 만들면 선형 대수 라이브러리를 활용해 효율을 높일 수 있다.  

>
    >im2col로 입력 데이터를 전개한 다음에는 합성곱 계층의 필터를 1열로 전개하고, 두 행렬의 곱을 계산하면 된다.  
    이는 전결합 계층에서 한것과 거의 같다.  

### 7-4.3 합성곱 계층 구현하기
>
    > 이 책에서는 im2col함수를 미리 만들어 제공한다. 사실 그 구현은 간단한 함수 10개 정도 묶은 것이다.

In [4]:
import sys, os 
sys.path.append(os.pardir)
from DLS_git_clone.common.util import im2col

x1 = np.random.rand(1,3,7,7)
col1 = im2col(x1,5,5,stride = 1, pad = 0)
print(col1.shape) 

x2 = np.random.rand(10,3,7,7)
col2 = im2col(x2,5,5,stride =1,pad = 0)
print(col2.shape)

(9, 75)
(90, 75)


>
    >여기에서는 두 가지 예를 보여주고 있다.  
      첫 번째 배치 크기가 1(데이터가 1개), 채널 3개 높이 너비가 7x7의 데이터이고,  
      두 번째는 배치 크기만 10이고 나머지는 첫번째와 동일하다.  
      im2co 함수를 적용한 두 경우 모두 2번째 차원의 원소는 75개이다.   
      이 값은 필터의 원소 수와 같다. 또한 배치 크기가 1일때 im2col의 결과의 크기가 (9,75)이고  
      10일 때는 그 10배인 (90,75) 크기의 데이터가 저장 된다.

>   
    > 이제 im2col을 사용하여 합성곱 계층을 구현해보자.

In [ ]:
class Convolution:
    def __init__(self,W,b,stride = 1, pad = 0):
        self.W = W
        self.b = b
        self.stride = stride 
        self.pad = pad
    
    def forward(self,x):
        FN,C,FH,FW = self.W.shpae
        N,C,H,W = x.shape
        out_h = int(1+(H+2*self.pad - FH) /  self.stride)
        out_w = int(1+(H+2*self.pad - FW) /  self.stride)
        
        col = im2col(x,FH,FW,self.stride,self,self.pad)
        col_W = self.W.reshape (FN,-1).T
        out = np.dot(col,col_W)+self.b
        
        out = out.reshape(N,out_h,out_w,-1).transpose(0,3,1,2)
        
        return out

>
    > 합성곱 계층은 필터, 편향 , 스트라이드, 패딩을 인수로 받아 초기화한다. 
    필터는 (FN,C,FH,FW)의 4차원 형상이다. 여기서 FN은 필터 개수, C는 채널, FH는 필터 높이,FW는 필터의 너비이다.  

>   
    > 앞의 합성곱 구현 코드에서 중요한 부분은 굴게 표시했다.  
    이 부분에서 입력 데이터를 im2col로 전개하고 필터도 reshape을 사용해 2차원 배열로 전개한다.  
    그리고 이렇게 전개한 두 행렬의 곱을 구한다.
>
    > 필터를 전개하는 부분은 그림 7-19에서 보듯 각 필터 블록을 1줄로 펼쳐 세운다.